In [1]:

import pandas as pd
import datetime as dt
import numpy as np
import cloudpickle
import matplotlib as mpl
from functools import reduce
import time
import warnings
import joblib as jl

warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras import initializers

import statsmodels.tsa as tsa
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import ensemble
from sklearn import linear_model
import sklearn.preprocessing as prep

In [3]:
# Naive (Benchmarks)

class naive():
    """
    Naive model implementation.
    
    Parameters
    ----------
        lag : int
            Number of days to go back in order to make forecast, e.g. day_lag=7 indicates
            a weekly persistent model, day_lag=1 indicates a daily persistent model, etc.
        period : str in ['D', 'Y']
    """
    def __init__(self, lag, period):
        self.lag = lag
        self.period = period
    
    def ingest_data(self, train_target, train_ida, train_planned, train_fuel_mix):
        self.data = train_target.copy()
       
    def train(self):
        # Date for which forecast will be made
        self.target_date = dt.datetime.combine(self.data.index.date[-1], dt.datetime.min.time()) + dt.timedelta(days=1)
    
    def forecast(self):
        # Make forecasts
        if self.period == 'D':
            forecast_df = self.data.loc[self.data.index.date == self.data.index.date[-(self.lag*24)]]
        elif self.period == 'Y':
            # Get corresponding date from lag years before    
            try:
                forecast_date = dt.datetime(self.target_date.year-1, self.target_date.month, self.target_date.day)
            except:
                forecast_date = dt.datetime(self.target_date.year-1, self.target_date.month, self.target_date.day-1)
            
            forecast_df = self.data.loc[self.data.index.date == forecast_date.date()]

        # Reindex forecasts to the appropriate forecast date and relabel forecasts_df
        forecast_df.index = pd.date_range(self.target_date, periods=24, freq='H')
        forecast_df.index.name = 'DeliveryPeriod'

        return(forecast_df)

In [2]:
# Random Forest

class random_forest():
    """
    Random forest model implementation.
    
    Parameters
    ----------
        model_params : dict
            n_estimators : int
                Number of trees
            max_depth : int
                Max tree depth
            max_features : int
                Number of features considered at each decision tree split
            n_jobs : int
                Number of processes (joblib argument for parallelisation)
        lag_params : dict
            price_lags : list of int
                Lags of day-ahead market price data to use as predictors
            ida_price_lags : list of int
                Lags of balancing market price data to use as predictors
            planned_lags : list of int
                Lags of forecast data (wind, demand, solar) to use as predictors
            
    """
    def __init__(self, model_params, lag_params):
        self.model = ensemble.RandomForestRegressor(**model_params, oob_score=True, random_state=1)
        self.price_lags = lag_params.get('price_lags', [])
        self.ida_price_lags = lag_params.get('ida_price_lags', [])
        self.planned_lags = lag_params.get('planned_lags', [])
        self.fuel_mix_lags = lag_params.get('fuel_mix_lags', [])
        self.gas_price_lags = lag_params.get('gas_price_lags', [])
        
    def ingest_data(self, train_price_data, train_ida, train_forecast, train_fuel_mix, train_gas_price):
        # Identify the latest start date among the datasets to align them
        start_dates = [df.index[0] for df in [train_price_data, train_ida, train_forecast, train_fuel_mix, train_gas_price] if not df.empty]
        latest_start_date = max(start_dates) if start_dates else None

        # Split into training predictors and test predictors
        last_day_of_data = dt.datetime.combine(train_forecast.index.date[-1], dt.datetime.min.time())
        test_index = pd.date_range(start=last_day_of_data, end=last_day_of_data + dt.timedelta(hours=23), freq='H')
    
        # Initialise predictors dataframe
        predictors = pd.DataFrame(index=train_forecast.index)
        
        # Build predictors from DAM prices
        for lag in self.price_lags:
            predictor_name = f"{train_price_data.columns[0]}-{lag}"
            predictors.insert(predictors.shape[1], predictor_name, train_price_data)
            predictors[predictor_name] = predictors[predictor_name].shift(lag)

        # Build predictors from Wind, Demand, and Solar forecasts
        for column in train_forecast:
            for lag in self.planned_lags:
                predictor_name = f"{column}-{lag}"
                predictors.insert(predictors.shape[1], predictor_name, train_forecast[column])
                predictors[predictor_name] = predictors[predictor_name].shift(lag)
                
        # Build predictors from IDA1, IDA2, and IDA3 prices
        for column in train_ida:
            for lag in self.ida_price_lags:
                predictor_name = f"{column}-{lag}"
                predictors.insert(predictors.shape[1], predictor_name, train_ida[column])
                predictors[predictor_name] = predictors[predictor_name].shift(lag)
                
        # Build predictors from Fuel mix data set
        for column in train_fuel_mix:
            for lag in self.fuel_mix_lags:
                predictor_name = f"{column}-{lag}"
                predictors.insert(predictors.shape[1], predictor_name, train_fuel_mix[column])
                predictors[predictor_name] = predictors[predictor_name].shift(lag)
                
        # Build predictors from Gas prices
        for column in train_gas_price:
            for lag in self.gas_price_lags:
                predictor_name = f"{column}-{lag}"
                predictors.insert(predictors.shape[1], predictor_name, train_gas_price[column])
                predictors[predictor_name] = predictors[predictor_name].shift(lag)

        # Split predictors into training and test predictors and store data for training and forecasting
        train_predictors = predictors.drop(test_index)
        test_predictors = predictors.loc[test_index, :]
        
        # Remove rows in training set with NAs
        notna_train_predictors_loc = train_predictors.notna().all(axis=1)
        train_predictors = train_predictors.loc[notna_train_predictors_loc]
        train_price_data = train_price_data.loc[notna_train_predictors_loc]
        
        # Store data into object
        self.train_predictors = train_predictors
        self.test_predictors = test_predictors
        self.train_price_data = train_price_data
        
        # Initialise dataframe for variable importances
        if not hasattr(self, 'variable_importances'):
            self.variable_importances = pd.DataFrame(index=train_predictors.columns)
        
    def train(self):
        # Fit the forecasting model
        self.model.fit(X=self.train_predictors, y=self.train_price_data['EURPrices'])
        
        # Store variable importances
        variable_importances = self.model.feature_importances_
        self.variable_importances.insert(self.variable_importances.shape[1], self.test_predictors.index.date[0], variable_importances)

    def forecast(self):
        forecast_df = pd.DataFrame(self.model.predict(self.test_predictors), index=self.test_predictors.index)
        forecast_df.index.name = 'DeliveryPeriod'
        return forecast_df



In [ ]:
# AR/ARX

"""
Parameters:
    model_params: dict with keys: lags, trend, ic, exog
    lag_params: dict with keys: ida_price_lags, planned_lags
"""
class ARX():
    """
    AR/ARX model implementation.
    
    Parameters
    ----------
        model_params : dict
            lags : list of int
                AR/ARX model orders to fit.
            trend : str in ['n', 'c', 't', 'ct']
                Specifies ar_model.AutoReg() parameter for model trend, if any.
            ic : str in ['aic', 'bic']
                Information criterion to use in determining the 'best' model order specification.
            exog : bool
                Whether or not to include exogenous data (i.e. implement ARX (True) or AR (False))
        lag_params : dict
            ida_price_lags : list of int
                Lags of intraday market price data to use as predictors
            planned_lags : list of int
                Lags of forecast, solar and wind data to use as predictors
    """
    def __init__(self, model_params, lag_params):
        self.lags = model_params['lags']
        self.trend = model_params['trend']
        self.ic = model_params['ic']
        self.exog = model_params['exog']
        
        self.ida_price_lags = lag_params['ida_price_lags']
        self.planned_lags = lag_params['planned_lags']
        self.fuel_mix_lags = lag_params['fuel_mix_lags']
        self.gas_price_lags = lag_params['gas_price_lags']

    def ingest_data(self, train_target, train_ida, train_planned, train_fuel_mix, train_gas_price):
        start_dates = [df.index[0] for df in [train_target, train_ida, train_planned, train_fuel_mix, train_gas_price]]
        latest_start_date = max(start_dates)

        # Split into training predictors and test predictors
        last_day_of_data = dt.datetime.combine(train_planned.index.date[-1], dt.datetime.min.time())
        test_index = pd.date_range(start=last_day_of_data, end=last_day_of_data+dt.timedelta(hours=23), freq='H')
    
        # Initialise predictors dataframe
        predictors = pd.DataFrame(index=train_planned.index)
        
        # Build predictors from IDA Prices (1,2 & 3)
        for column in train_ida:
            for lag in self.ida_price_lags:
                predictor_name = f"{column}-{lag}"
                predictors.insert(predictors.shape[1], predictor_name, train_ida[column])
                predictors[predictor_name] = predictors[predictor_name].shift(lag)

        # Build predictors from Wind, Solar and Demand
        for column in train_planned:
            for lag in self.planned_lags:
                predictor_name = f"{column}-{lag}"
                predictors.insert(predictors.shape[1], predictor_name, train_planned[column])
                predictors[predictor_name] = predictors[predictor_name].shift(lag)
                
         # Build predictors from Fuel Mix
        for column in train_fuel_mix:
            for lag in self.fuel_mix_lags:
                predictor_name = f"{column}-{lag}"
                predictors.insert(predictors.shape[1], predictor_name, train_fuel_mix[column])
                predictors[predictor_name] = predictors[predictor_name].shift(lag)
       
        # Build predictors from Gas prices
        for column in train_gas_price:
            for lag in self.gas_price_lags:
                predictor_name = f"{column}-{lag}"
                predictors.insert(predictors.shape[1], predictor_name, train_gas_price[column])
                predictors[predictor_name] = predictors[predictor_name].shift(lag)

        # Split predictors into training and test predictors and store data for training and forecasting
        train_predictors = predictors.drop(test_index)
        test_predictors = predictors.loc[test_index,:]
        
        # Remove rows in training set with NAs
        notna_train_predictors_loc = train_predictors.notna().all(axis=1)
        train_predictors = train_predictors.loc[notna_train_predictors_loc]
        train_target = train_target.loc[notna_train_predictors_loc]
        
        # Store data into object
        self.train_predictors = train_predictors
        self.test_predictors = test_predictors
        self.train_target = train_target
        
        # Initialise dataframe for variable importances
#         if not hasattr(self, "variable_importances"):
#             self.variable_importances = pd.DataFrame(index=train_predictors.columns)
        
   
    def train(self):
        # Fit AR/ARX models with different lag values
        if self.exog:
            model_selection = jl.Parallel(n_jobs=-1, backend='threading') \
                (jl.delayed(tsa.ar_model.AutoReg(self.train_target.values,lag,self.trend,exog=self.train_predictors.values).fit)() for lag in self.lags)
        else:
            model_selection = jl.Parallel(n_jobs=-1, backend='threading') \
                (jl.delayed(tsa.ar_model.AutoReg(self.train_target.values,lag,self.trend).fit)() for lag in self.lags)
        
        # Pick best model based on the specified information criterion (AIC or BIC)
        ar_ics = [model.aic for model in model_selection] if self.ic=='aic' else [model.bic for model in model_selection]
        self.model = model_selection[ar_ics.index(min(ar_ics))]
        
    def forecast(self):
        # Make forecasts
        if self.exog:
            forecast = self.model.predict(start=self.train_target.shape[0], end=self.train_target.shape[0]+23, exog_oos=self.test_predictors)
        else:
            forecast = self.model.predict(start=self.train_target.shape[0], end=self.train_target.shape[0]+23)
        
        # Store forecasts in labelled dataframe
        forecast_df = pd.DataFrame(dict(Forecast=forecast), index=self.test_predictors.index)
        return(forecast_df)

In [ ]:
# Artificial Neural Networks

def create_ffnn(num_of_nodes, input_cols, act_fn, n_layers=3, opt='adam', loss='mse'):
    """
    Create a Keras neural network model (composed of Dense feedforward layers).
    
    Parameters
    ----------
        num_of_nodes : int
            Number of neurons per layer.
        input_cols : int
            Number of features/predictors.
        act_fn : str in ['tanh', 'sigmoid', 'relu']
            Activation function.
        n_layers : int, default=3
            Number of hidden layers.
        opt : str, default='adam'
            Optimiser for training.
        loss : str, default='mse'
            Loss function for training.
    Returns
    -------
        model : tensorflow.python.keras.engine.sequential.Sequential
            Sequential model representing the neural network model.
    """
    # Initialise neural network model
    model = Sequential()
    initializer = initializers.he_uniform(1)
    
    # Add first hidden layer (with input layer specification)
    model.add(Dense(num_of_nodes, activation=act_fn, input_shape=(input_cols,), kernel_initializer=initializer))
    
    # Add remaining hidden layers
    for _ in range(n_layers-1):
        model.add(Dense(num_of_nodes, activation=act_fn, kernel_initializer=initializer))

    # Add output layer
    model.add(Dense(1, kernel_initializer=initializer))

    # Configure model optimizer and loss function
    model.compile(optimizer=opt, loss=loss)

    return(model)


def scale_predictors(predictors, activation, copy_df=True):
    """
    Rescale a dataframe of predictors according to the given activation function.
    
    Parameters
    ----------
        predictors : pandas.DataFrame
        activation : str in ['tanh', 'sigmoid', 'relu']
        
    Returns
    -------
        scaler : sklearn.preprocessing._data.MinMaxScaler
        scaled_predictors : pandas.DataFrame
    """
    activation_ranges = {
        'tanh': (-1,1),
        'sigmoid': (0,1),
        'relu': (0,5)
    }
    # Initialise scaler
    scaler = prep.MinMaxScaler(feature_range=activation_ranges[activation], copy=copy_df)
    
    # Fit scaler and scale the data
    scaled_predictors = pd.DataFrame(scaler.fit_transform(predictors), index=predictors.index, columns=predictors.columns)
    
    return(scaler, scaled_predictors)


class ffnn():
    """
    Feedforward neural network model implementation.
    
    Parameters
    ----------
        model_params : dict
            init_params : dict
                Arguments to pass into create_ffnn() function
            train_params : dict
                Extra arguments to pass into Sequential.fit() function
            other_params : dict
                Other extra arguments (number of epochs and bool to specify whether to use GPU)
        lag_params : dict
            price_lags : list of int
                Lags of day-ahead market price data to use as predictors
            ida_price_lags : list of int
                Lags of balancing market price data to use as predictors
            planned_lags : list of int
                Lags of forecast and wind data to use as predictors
    """
    def __init__(self, model_params, lag_params):
        self.init_params = model_params['init_params']
        self.train_params = model_params['train_params']
        self.other_params = model_params['other_params']
        
        # Set to GPU/CPU
        self.device = '/device:GPU:0' if self.other_params['GPU'] else '/CPU:0'
        
        if not hasattr(self, 'model'):
            self.model = create_ffnn(**self.init_params)
        
        self.price_lags = lag_params['price_lags']
        self.ida_price_lags = lag_params['ida_price_lags']
        self.planned_lags = lag_params['planned_lags']
        self.fuel_mix_lags = lag_params['fuel_mix_lags']
        self.gas_price_lags = lag_params['gas_price_lags']
        
    def ingest_data(self, train_target, train_ida, train_planned, train_fuel_mix, train_gas_price):
        start_dates = [df.index[0] for df in [train_target, train_ida, train_planned, train_fuel_mix, train_gas_price]]
        latest_start_date = max(start_dates)

        # Split into training predictors and test predictors
        last_day_of_data = dt.datetime.combine(train_planned.index.date[-1], dt.datetime.min.time())
        test_index = pd.date_range(start=last_day_of_data, end=last_day_of_data+dt.timedelta(hours=23), freq='H')

        # Initialise predictors dataframe
        predictors = pd.DataFrame(index=train_planned.index)

        # Build predictors from EURPrices
        for lag in self.price_lags:
            predictor_name = f"{train_target.columns[0]}-{lag}"
            predictors.insert(predictors.shape[1], predictor_name, train_target)
            predictors[predictor_name] = predictors[predictor_name].shift(lag)
        
        #print(f"Predictors after adding price lags: {predictors.shape}")

        # Build predictors from IDA Priceprint(f"Predictors after adding IDA lags: {predictors.shape}")
        for column in train_ida:
            for lag in self.ida_price_lags:
                predictor_name = f"{column}-{lag}"
                predictors.insert(predictors.shape[1], predictor_name, train_ida[column])
                predictors[predictor_name] = predictors[predictor_name].shift(lag)
        
        #print(f"Predictors after adding IDA lags: {predictors.shape}")
                
        # Build predictors from Wind, solar and Demand
        for column in train_planned:
            for lag in self.planned_lags:
                predictor_name = f"{column}-{lag}"
                predictors.insert(predictors.shape[1], predictor_name, train_planned[column])
                predictors[predictor_name] = predictors[predictor_name].shift(lag)
        
        #print(f"Predictors after adding planned lags: {predictors.shape}")
        
        # Build predictors from Fuel Mix
        for column in train_fuel_mix:
            for lag in self.fuel_mix_lags:
                predictor_name = f"{column}-{lag}"
                predictors.insert(predictors.shape[1], predictor_name, train_fuel_mix[column])
                predictors[predictor_name] = predictors[predictor_name].shift(lag)
       
        # Build predictors from Gas prices
        for column in train_gas_price:
            for lag in self.gas_price_lags:
                predictor_name = f"{column}-{lag}"
                predictors.insert(predictors.shape[1], predictor_name, train_gas_price[column])
                predictors[predictor_name] = predictors[predictor_name].shift(lag)
                
        # Split predictors into training and test predictors and store data for training and forecasting
        train_predictors = predictors.drop(test_index)
        test_predictors = predictors.loc[test_index,:]
        
        #print(f"Training predictors shape: {train_predictors.shape}")
        #print(f"Test predictors shape: {test_predictors.shape}")

        # Remove rows in training set with NAs
        notna_train_predictors_loc = train_predictors.notna().all(axis=1)
        train_predictors = train_predictors.loc[notna_train_predictors_loc]
        train_target = train_target.loc[notna_train_predictors_loc]
        
        #print(f"Training predictors shape after dropping NAs: {train_predictors.shape}")

        # Scale predictors and target, and store in object
        self.train_predictors_scaler, self.scaled_train_predictors = scale_predictors(train_predictors, activation=self.init_params['act_fn'])
        self.train_target_scaler, self.scaled_train_target = scale_predictors(train_target, activation=self.init_params['act_fn'])
        self.scaled_test_predictors = self.train_predictors_scaler.transform(test_predictors)
        
        self.test_index = test_predictors.index
        
        # Initialise dataframe for variable importances
        if not hasattr(self, 'variable_importances'):
            self.variable_importances = pd.DataFrame(index=train_predictors.columns)
            
        # Calculate total number of features
        total_features = (1 * len(self.price_lags)) + (train_ida.shape[1] * len(self.ida_price_lags)) + (train_planned.shape[1] * len(self.planned_lags)) + (train_fuel_mix.shape[1] * len(self.fuel_mix_lags)) + (train_gas_price.shape[1] * len(self.gas_price_lags))
    
        # Verify the input shape matches total features
        assert self.scaled_train_predictors.shape[1] == total_features, f"Expected input shape of {total_features}, but got {self.scaled_train_predictors.shape[1]}"

        # Initialise dataframe for variable importances
        if not hasattr(self, 'variable_importances'):
            self.variable_importances = pd.DataFrame(index=train_predictors.columns)

        # Dynamically set the input_cols parameter in the model
        self.model = create_ffnn(self.init_params['num_of_nodes'], total_features, self.init_params['act_fn'], self.init_params['n_layers'], self.init_params['opt'], self.init_params['loss'])

    def train(self):
        # Fit the forecasting model
        if hasattr(self, 'history'):
            self.history = self.model.fit(x=self.scaled_train_predictors.values, y=self.scaled_train_target.values, epochs=self.other_params['subseq_epochs'], batch_size=len(self.scaled_train_target), **self.train_params)
        else:
            self.history = self.model.fit(x=self.scaled_train_predictors.values, y=self.scaled_train_target.values, epochs=self.other_params['init_epochs'], batch_size=len(self.scaled_train_target), **self.train_params)
        
    def forecast(self):
        forecast = self.train_target_scaler.inverse_transform(self.model.predict(x=self.scaled_test_predictors))
        forecast_df = pd.DataFrame(forecast, index=self.test_index)
        return(forecast_df)

In [ ]:
# Recurrent Neural Networks

def create_data_window(data, n_steps, step_size=24):
    """
    Given a (single column) predictor dataframe, returns a dataframe where each column is a lagged version
    of the original column. This is part of the pre-processing step for reformatting data as RNN input/s.
    
    Parameters
    ----------
        data : pandas.DataFrame
        n_steps : int
            Maximum number of steps to shift back. 
        step_size : int, default=24
    """
    new_data = data.copy()
    column = data.columns[0]

    # Add lagged values as new columns
    for step in range(n_steps):
        new_data.insert(new_data.shape[1], f"{column}-{step_size*(step+1)}h", new_data[[column]].shift(step_size * (step+1)))
    
    return(new_data)


def get_rnn_scaler(predictors, activation, copy_df=True):
    """
    Given a dataframe, return a scaler that can be used on other dataframes. This function is used
    for the predictors. The scaler is fit on (and used to transform) the train predictors and then
    used to transform the test predictors.
    
    Parameters
    ----------
        predictors : pandas.DataFrame
        activation : str in ['tanh', 'sigmoid', 'relu']
        copy_df : bool, default=True
            Set to False to perform inplace row normalization and avoid a
            copy (if the input is already a numpy array).
            
    Returns
    -------
        scaler : sklearn.preprocessing._data.MinMaxScaler
    """
    activation_ranges = {
        'tanh': (-1,1),
        'sigmoid': (0,1),
        'relu': (0,5)
    }
    real_predictors = predictors[:,0,:].copy()
    
    # Create and fit scaler
    scaler = prep.MinMaxScaler(feature_range=activation_ranges[activation], copy=copy_df)
    scaler.fit(real_predictors)
    
    return(scaler)


def transform_rnn_scale(predictors, scaler):
    """
    Function to apply the scaler to the array of predictors. This function is needed instead
    of the usual scaler.transform() method because predictors is a 3D numpy array instead
    of the 2D dataframe/array expected by the function.
    
    Parameters
    ----------
        predictors : numpy.array
        scaler : sklearn.preprocessing._data.MinMaxScaler
        
    Returns
    -------
        scaled_predictors : numpy.array
    """
    # Transform predictors
    scaled_predictors = predictors.copy()
    
    for time_step in range(scaled_predictors.shape[1]):
        scaled_predictors[:,time_step,:] = scaler.transform(scaled_predictors[:,time_step,:])
        
    return(scaled_predictors)


def create_rnn(num_of_blocks, n_timesteps, n_features, act_fn, n_layers=3, opt='adam', loss='mse'):
    """
    Create a Keras neural network model (composed of LSTM layers).
    
    Parameters
    ----------
        num_of_blocks : int
            Number of neurons per layer.
        input_cols : int
            Number of features/predictors (excluding lagged predictors).
        act_fn : str in ['tanh', 'sigmoid', 'relu']
            Activation function.
        n_layers : int, default=3
            Number of hidden layers.
        opt : str, default='adam'
            Optimiser for training.
        loss : str, default='mse'
            Loss function for training.
    Returns
    -------
        model : tensorflow.python.keras.engine.sequential.Sequential
            Sequential model representing the neural network model.
    """
    model = Sequential()

    # Add first hidden layer
    if n_layers == 1:
        model.add(LSTM(num_of_blocks, activation=act_fn, input_shape=(n_timesteps, n_features)))
    else:
        model.add(LSTM(num_of_blocks, return_sequences=True, activation=act_fn, input_shape=(n_timesteps, n_features)))
    
    # Add remaining hidden layers
    for n in range(n_layers-1):
        if n == n_layers-2:
            model.add(LSTM(num_of_blocks, activation=act_fn))
        else:
            model.add(LSTM(num_of_blocks, return_sequences=True, activation=act_fn))

    # Add output layer
    model.add(Dense(1))#, kernel_initializer=initializer))

    # Configure model optimizer and loss function
    model.compile(optimizer=opt, loss=loss)

    return(model)


class rnn():
    """
    Recurrent neural network (LSTM) model implementation.
    
    Parameters
    ----------
        model_params : dict
            init_params : dict
                Arguments to pass into create_rnn() function
            train_params : dict
                Extra arguments to pass into Sequential.fit() function
            other_params : dict
                Other extra arguments (number of epochs and bool to specify whether to use GPU)
    """
    def __init__(self, model_params, lag_params):
        self.init_params = model_params['init_params']
        self.train_params = model_params['train_params']
        self.other_params = model_params['other_params']
        
        # Set to GPU/CPU
        self.device = '/device:GPU:0' if self.other_params['GPU'] else '/CPU:0'
        
        if not hasattr(self, 'model'):
            self.model = create_rnn(**self.init_params)
        
    def ingest_data(self, train_target, train_ida, train_planned, train_fuel_mix):
        # Get the the latest start date of the variables
        start_dates = [df.index[0] for df in [train_target, train_ida, train_planned]]
        latest_start_date = max(start_dates)
        target_end_date = train_target.index[-1]

        # Split up planned data into its component columns
        train_wind = train_planned.loc[:,'Wind'].to_frame()
        train_demand = train_planned.loc[:,'Demand'].to_frame()
        train_solar = train_planned.loc[:,'Solar'].to_frame()
        
        # Split up IDA data into its component columns
        train_ida1 = train_ida.loc[:,'IDA1_Price'].to_frame()
        train_ida2 = train_ida.loc[:,'IDA2_Price'].to_frame()
        train_ida3 = train_ida.loc[:,'IDA3_Price'].to_frame()
        
        # Add extra rows corresponding to forecast dates
        empty_frame = pd.DataFrame(index=train_planned.index[-24:])
        train_target = pd.concat([train_target, empty_frame])
        train_ida1 = pd.concat([train_ida1, empty_frame])
        train_ida2 = pd.concat([train_ida2, empty_frame])
        train_ida3 = pd.concat([train_ida3, empty_frame])
        
        # Reformat data for input into RNNs
        n_timesteps = self.init_params['n_timesteps']
        rnn_train_prices = create_data_window(train_target, n_timesteps)

        rnn_train_target = rnn_train_prices[[rnn_train_prices.columns[0]]]
        rnn_train_prices.drop(rnn_train_prices.columns[0], axis=1, inplace=True)

        rnn_train_ida1 = create_data_window(train_ida1, n_timesteps)
        rnn_train_ida1.drop(rnn_train_ida1.columns[0], axis=1, inplace=True)
        
        rnn_train_ida2 = create_data_window(train_ida2, n_timesteps)
        rnn_train_ida2.drop(rnn_train_ida2.columns[0], axis=1, inplace=True)
        
        rnn_train_ida3 = create_data_window(train_ida3, n_timesteps)
        rnn_train_ida3.drop(rnn_train_ida3.columns[0], axis=1, inplace=True)

        rnn_train_wind = create_data_window(train_wind, n_timesteps)
        rnn_train_wind.drop(rnn_train_wind.columns[-1], axis=1, inplace=True)

        rnn_train_demand = create_data_window(train_demand, n_timesteps)
        rnn_train_demand.drop(rnn_train_demand.columns[-1], axis=1, inplace=True)
        
        rnn_train_solar = create_data_window(train_solar, n_timesteps)
        rnn_train_solar.drop(rnn_train_solar.columns[-1], axis=1, inplace=True)
               
        # Remove all rows with NAs that arose due to the reformatting above
        rnn_train_prices = rnn_train_prices.loc[rnn_train_prices.notna().all(axis=1)]
        rnn_train_target = rnn_train_target.loc[rnn_train_target.notna().all(axis=1)]
        rnn_train_ida1 = rnn_train_ida1.loc[rnn_train_ida1.notna().all(axis=1)]
        rnn_train_ida2 = rnn_train_ida2.loc[rnn_train_ida2.notna().all(axis=1)]
        rnn_train_ida3 = rnn_train_ida3.loc[rnn_train_ida3.notna().all(axis=1)]
        rnn_train_wind = rnn_train_wind.loc[rnn_train_wind.notna().all(axis=1)]
        rnn_train_demand = rnn_train_demand.loc[rnn_train_demand.notna().all(axis=1)]
        rnn_train_solar = rnn_train_solar.loc[rnn_train_solar.notna().all(axis=1)]
        
        # Match data start dates (again)
        #min_start_index = max(rnn_train_prices.index[0], rnn_train_ida1.index[0], rnn_train_ida2.index[0], rnn_train_ida3.index[0], rnn_train_wind.index[0], rnn_train_demand.index[0], rnn_train_solar.index[0])
        rnn_train_target = rnn_train_target.loc[rnn_train_target.index >= rnn_train_prices.index[0]]
        rnn_train_ida1 = rnn_train_ida1.loc[rnn_train_ida1.index >= rnn_train_prices.index[0]]
        rnn_train_ida2 = rnn_train_ida2.loc[rnn_train_ida2.index >= rnn_train_prices.index[0]]
        rnn_train_ida3 = rnn_train_ida3.loc[rnn_train_ida3.index >= rnn_train_prices.index[0]]
        rnn_train_wind = rnn_train_wind.loc[rnn_train_wind.index >= rnn_train_prices.index[0]]
        rnn_train_demand = rnn_train_demand.loc[rnn_train_demand.index >= rnn_train_prices.index[0]]
        rnn_train_solar = rnn_train_solar.loc[rnn_train_solar.index >= rnn_train_prices.index[0]]
        
        common_indices = rnn_train_prices.index.intersection(rnn_train_ida1.index).intersection(rnn_train_ida2.index).intersection(rnn_train_ida3.index).intersection(rnn_train_wind.index).intersection(rnn_train_demand.index).intersection(rnn_train_solar.index)
        rnn_train_target = rnn_train_target.loc[rnn_train_target.index.isin(common_indices)]
        rnn_train_prices = rnn_train_prices.loc[rnn_train_prices.index.isin(common_indices)]
        rnn_train_ida1 = rnn_train_ida1.loc[rnn_train_ida1.index.isin(common_indices)]
        rnn_train_ida2 = rnn_train_ida2.loc[rnn_train_ida2.index.isin(common_indices)]
        rnn_train_ida3 = rnn_train_ida3.loc[rnn_train_ida3.index.isin(common_indices)]
        rnn_train_wind = rnn_train_wind.loc[rnn_train_wind.index.isin(common_indices)]
        rnn_train_demand = rnn_train_demand.loc[rnn_train_demand.index.isin(common_indices)]
        rnn_train_solar = rnn_train_solar.loc[rnn_train_solar.index.isin(common_indices)]
        
        # Split into training and test set
        self.test_index = rnn_train_prices.index[-24:]

        # Test data
        rnn_test_prices = rnn_train_prices.loc[rnn_train_prices.index >= self.test_index[0]]
        rnn_test_ida1 = rnn_train_ida1.loc[rnn_train_ida1.index >= self.test_index[0]]
        rnn_test_ida2 = rnn_train_ida2.loc[rnn_train_ida2.index >= self.test_index[0]]
        rnn_test_ida3 = rnn_train_ida3.loc[rnn_train_ida3.index >= self.test_index[0]]
        rnn_test_wind = rnn_train_wind.loc[rnn_train_wind.index >= self.test_index[0]]
        rnn_test_demand = rnn_train_demand.loc[rnn_train_demand.index >= self.test_index[0]]
        rnn_test_solar = rnn_train_solar.loc[rnn_train_solar.index >= self.test_index[0]]

        # Train data
        rnn_train_prices = rnn_train_prices.loc[rnn_train_prices.index < self.test_index[0]]
        rnn_train_ida1 = rnn_train_ida1.loc[rnn_train_ida1.index < self.test_index[0]]
        rnn_train_ida2 = rnn_train_ida2.loc[rnn_train_ida2.index < self.test_index[0]]
        rnn_train_ida3 = rnn_train_ida3.loc[rnn_train_ida3.index < self.test_index[0]]
        rnn_train_wind = rnn_train_wind.loc[rnn_train_wind.index < self.test_index[0]]
        rnn_train_demand = rnn_train_demand.loc[rnn_train_demand.index < self.test_index[0]]
        rnn_train_solar = rnn_train_solar.loc[rnn_train_solar.index < self.test_index[0]]
        rnn_train_target = rnn_train_target.loc[rnn_train_target.index < self.test_index[0]]
        
        # Combine train and test features into one tensor each
        rnn_test_predictors = np.hstack((rnn_test_prices, rnn_test_ida1, rnn_test_ida2, rnn_test_ida3, rnn_test_wind, rnn_test_demand, rnn_test_solar)).reshape(rnn_test_prices.shape[0], 7, n_timesteps).transpose(0,2,1)
        rnn_train_predictors = np.hstack((rnn_train_prices, rnn_train_ida1, rnn_train_ida2, rnn_train_ida3, rnn_train_wind, rnn_train_demand, rnn_train_solar)).reshape(rnn_train_prices.shape[0], 7, n_timesteps).transpose(0,2,1)

        # Scale predictors and target, and store in object
        self.rnn_train_predictors_scaler = get_rnn_scaler(rnn_train_predictors, activation=self.init_params["act_fn"])
        self.rnn_scaled_train_predictors = transform_rnn_scale(rnn_train_predictors, self.rnn_train_predictors_scaler)
        self.rnn_scaled_test_predictors = transform_rnn_scale(rnn_test_predictors, self.rnn_train_predictors_scaler)
        self.rnn_train_target_scaler = get_rnn_scaler(rnn_train_target.values.reshape(-1, 1, 1), activation=self.init_params['act_fn'])
        self.rnn_scaled_train_target = self.rnn_train_target_scaler.transform(rnn_train_target.values.reshape(-1, 1))


    def train(self):
        with tf.device(self.device):
            epochs = self.other_params['subseq_epochs'] if hasattr(self, 'history') else self.other_params['init_epochs']
            self.history = self.model.fit(x=self.rnn_scaled_train_predictors, y=self.rnn_scaled_train_target, epochs=epochs, **self.train_params)

    def forecast(self):
        forecast = self.rnn_train_target_scaler.inverse_transform(self.model.predict(x=self.rnn_scaled_test_predictors))
        forecast_df = pd.DataFrame(forecast, index=self.test_index)
        return forecast_df